In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import pdb
import time
from random import randint


In [2]:
years = range(2000,2022)
months = ['04', '10']
home_page = 'https://www.churchofjesuschrist.org'
lang = 'rus'

var_names = ['year', 'month', 'speaker', 'title', 'text']
conf_data = []

for year in years:
    for month in months:
        start_time_month = time.time()
        
        # Download conference html file for given date
        conf_page = requests.get(f'{home_page}/study/general-conference/{year}/{month}?lang={lang}')
        conf_soup = BeautifulSoup(conf_page.content, 'html.parser')
        
        # Information for each talk in given conference
        links = conf_soup.find_all(class_="item-3cCP7")
        for link_contents in links:
            try:
                speaker_name = link_contents.find(class_='subtitle-MuO4X')
                if speaker_name is None:
                    continue
                speaker_name = speaker_name.string
                subdir = link_contents.get('href')
                
                # Skip talk if there is no html site with talk text (usually general women's meeting)
                if re.search('media', subdir) != None:
                    print(f'Missing talk text: {speaker_name}, {month} {year}')
                    conf_data.append((year, month, speaker_name, None, None))
                    continue
            except:
                print('Exception: Problem getting speaker name')
                print(f'{month} {year} Conference')
                print(link_contents)
                raise
                
            title_obj = link_contents.find('div', class_="itemTitle-23vMm")
            title = title_obj.find('p').getText()
            #speaker = talk_soup.find('p', id = 'p1').getText()
#             print(f"\nCurrent Talk: {title}")
#             print(f"Date: {month}, {year}")
#             print(f"Speaker: {speaker_name}")
            
            # Refreshing page up to 3 times if an error occurs
            talk_loaded = False
            iterations = 0
            MAX_ITER = 3
            while talk_loaded == False and iterations < MAX_ITER:
                # Get talk text
                talk_page = requests.get(f'{home_page}{subdir}')
                talk_soup = BeautifulSoup(talk_page.content, 'html.parser')
                talk_title = talk_soup.title.string
                #speaker_name_lang = talk_soup.find('p', id = 'p1').getText()
        
                # Check that GET request was successful
                if iterations == MAX_ITER:
                    print(f'Talk failed loading {MAX_ITER} times\nAborting')
                    raise
                elif talk_title == 'Service Not Available':
                    print(f'WARNING: {month} {year} conference talk by {speaker_name} did not load--{talk_page}')
                    iterations += 1
                    time.sleep(randint(2,5))
                else:
                    talk_loaded = True
            
            # Replace lettered references with reference content
#             references = talk_soup.find_all(id=re.compile('note[0-9]+'))
            
#             # Combine references that are from same note
#             new_references = []
#             note = ''
#             for ref in references:
#                 # If this is the beginning of a section of a new footnote, append as new footnote
#                 new_string = ' '.join(ref.stripped_strings)
#                 next_note = re.search('note[0-9]+', ref.get('id')).group(0)
#                 # If the next section is part of the same footnote, add to same footnote
#                 if note == next_note:
#                     new_references[-1] = ' '.join([new_references[-1], new_string])
#                 else:
#                     new_references.append(new_string)
#                 note = next_note
            
#             references = new_references
            
#             # Replace number in talk body text with formatted citation, be it scripture or other reference
#             talk_refs = talk_soup.find_all(href=re.compile('note[0-9]+'))
#             for i, new_string in enumerate(references):
#                 try:
#                     talk_refs[i].string.replace_with(f' ({new_string}) ')
#                 except:
#                     print('Exception: Problem replacing reference string')
#                     print(f'{month} {year} Conference')
#                     print(f'Speaker: {speaker_name}, Talk: {talk_title}')
#                     print(f'Talk total references: {len(references)}, Currently on: {i}')
#                     raise
            
            # Get talk body text after adding references
            try:
                talk_text = talk_soup.find(class_='body-block').get_text(separator=' ', strip=True)
            except:
                print('Exception: Problem obtaining body text')
                print(f'{month} {year} Conference')
                print(f'Speaker: {speaker_name}, Talk: {talk_title}')
                continue
            conf_data.append((year, month, speaker_name, talk_title, talk_text))
        
        # Compute and report data scraping time for each conference session
        time_min_month = round( (time.time()-start_time_month)/60, 2)
        print(f'Extracted {month} {year} conference talks in {time_min_month} minutes\n')
            
conf_df = pd.DataFrame(conf_data, columns=var_names)
conf_df.to_csv(f'conference_{lang}.csv', index=False)

Extracted 04 2000 conference talks in 0.26 minutes

Extracted 10 2000 conference talks in 0.01 minutes

Extracted 04 2001 conference talks in 0.01 minutes

Extracted 10 2001 conference talks in 0.17 minutes

Extracted 04 2002 conference talks in 2.34 minutes

Exception: Problem obtaining body text
10 2002 Conference
Speaker: James E. Faust, Talk: Error
Extracted 10 2002 conference talks in 0.46 minutes

Extracted 04 2003 conference talks in 0.46 minutes

Extracted 10 2003 conference talks in 0.01 minutes

Exception: Problem obtaining body text
04 2004 Conference
Speaker: David B. Haight, Talk: Неизвестная ошибка
Extracted 04 2004 conference talks in 0.75 minutes

Extracted 10 2004 conference talks in 0.17 minutes

Extracted 04 2005 conference talks in 0.26 minutes

Extracted 10 2005 conference talks in 0.17 minutes

Extracted 04 2006 conference talks in 0.19 minutes

Extracted 10 2006 conference talks in 0.34 minutes

Extracted 04 2007 conference talks in 0.36 minutes

Extracted 10 200

In [3]:
conf_df

,year,month,speaker,title,text
0,2000,04,Gordon B. Hinckley,Свидетельство всему миру,"Мои возлюбленные братья и сестры, какое велико..."
1,2000,04,Boyd K. Packer,Разделяющиеся языки . . . огненные,"Перед теми из нас, кому поручено здесь выступа..."
2,2000,04,Harold G. Hillam,Будущие руководители,Во время прошлой Генеральной конференции мое в...
3,2000,04,Coleen K. Menlove,Жить-поживать да добра наживать,"Дети любят сказки. Когда я была маленькой, я м..."
4,2000,04,Dallin H. Oaks,Воскресение,В книге Иова задан глобальный вопрос: “Когда у...
...,...,...,...,...,...
1522,2021,04,Тимоти Дж. Дайчес,Свет придерживается света,"Мои дорогие братья и сестры, я радуюсь вместе ..."
1523,2021,04,Д. Тодд Кристоферсон,Почему мы следуем путем заветов,На протяжении всего своего служения Президент ...
1524,2021,04,Алан Р. Уолкер,Людям свет Евангелия дан,Прекрасный гимн Святых последних дней «Голос Н...
1525,2021,04,Дэвид A. Беднар,«Принципы Евангелия Моего»,В октябре 1849 года на Генеральной конференции...


[<p><span>Генеральная конференция, апрель 2021 года</span></p>,
 <p><span>Содержание</span></p>,
 <p><span>Субботняя утренняя сессия</span></p>,
 <p><span>Приветственное послание</span></p>,
 <p class="subtitle-MuO4X">Рассел М. Нельсон</p>,
 <p><span>Бог среди нас</span></p>,
 <p class="subtitle-MuO4X">Дитер Ф. Ухтдорф</p>,
 <p><span>Важные беседы</span></p>,
 <p class="subtitle-MuO4X">Джой Д. Джонс</p>,
 <p><span>Обучать по примеру Спасителя</span></p>,
 <p class="subtitle-MuO4X">Джен Э. Ньюмен</p>,
 <p><span>Сердца, связанные вместе</span></p>,
 <p class="subtitle-MuO4X">Гэри И. Стивенсон</p>,
 <p><span>Место в гостинице</span></p>,
 <p class="subtitle-MuO4X">Геррит У. Гонг</p>,
 <p><span>На храм смотреть люблю я</span></p>,
 <p class="subtitle-MuO4X">Генри Б. Айринг</p>,
 <p><span>Субботняя дневная сессия</span></p>,
 <p><span>Поддержка голосованием представителей Высшей власти, региональных представителей Кворумов Семидесяти, а также высших должностных лиц</span></p>,
 <p class="su